In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import sys
from tqdm import tqdm
import torch.nn.functional as F
import math
import random
import h5py
# Import utils and models
import utils, models

print("Is PyTorch using GPU?", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float32

In [ ]:
print("Loading data...")
# Import data
hf = h5py.File("Data/GP_galactic_data.h5", "r")

X_training = torch.tensor( np.array(hf.get("X_training")), dtype = DTYPE )
Y_training = torch.tensor( np.array(hf.get("Y_training")), dtype = DTYPE )

X_val = torch.tensor( np.array(hf.get("X_val")), dtype = DTYPE )
Y_val = torch.tensor( np.array(hf.get("Y_val")), dtype = DTYPE )

X_test = torch.tensor( np.array(hf.get("X_test")), dtype = DTYPE )
Y_test = torch.tensor( np.array(hf.get("Y_test")), dtype = DTYPE )

hf.close()

print("Training shape: ", X_training.shape, Y_training.shape)
print("Val shape: ", X_val.shape, Y_val.shape)
print("Test shape: ", X_test.shape, Y_test.shape)

In [ ]:
batch_size = 5
n_batches = int( X_test.shape[0] / batch_size )

labeling_order = torch.unique(Y_test, return_counts = False).type(torch.int).detach().cpu().numpy().tolist()
print( "labeling_order: ", labeling_order )

# Compute weights from frequency of classes in the training set
unique_training, counts_training = torch.unique( Y_test, return_counts = True )

inverse_freqs =  1 / counts_training
weights_tensor = ( inverse_freqs / torch.sum(inverse_freqs) ).to(device)

# Define model
input_dim = 6
n_classes = len(labeling_order)
d_model = 32
nhead = 4
num_layers = 4

model = models.TransformerClassifier(input_dim = input_dim, n_classes = n_classes, d_model = d_model, nhead = nhead, num_layers = num_layers, weights_tensor = weights_tensor, DTYPE = DTYPE, device = device, labeling_order = labeling_order).to(device)

In [ ]:
# Load weights
model.load_state_dict( torch.load("Different_Tests/Test_1/Models_Data/Model_Saved/model_epoch_61.pt") )

In [ ]:
y_true_list, y_pred_list = [], []

for n in tqdm(range(n_batches)):
    y_pred_batch = model(X_test[n*batch_size:(n+1)*batch_size,0,:,:].to(device))
    y_pred_batch = torch.tensor([labeling_order[i] for i in torch.argmax(y_pred_batch, dim = 1).detach().cpu().numpy().tolist()], dtype = DTYPE).view(-1,1)
    y_true = Y_test[n*batch_size:(n+1)*batch_size]
    y_true_list.append(y_true)
    y_pred_list.append(y_pred_batch)

In [ ]:
y_pred_list

In [ ]:
model(X_training[n*batch_size:(n+1)*batch_size,0,:,:].to(device))

In [ ]:
torch.unique(Y_test, return_counts = True)

In [ ]:
y_true